# working notebook

### Utility Functions:
- load_bitcoin_edge_data(filename)
- user_activity_dataframe(bitcoin_df) <-- uses user_stats(bitcoin_df)
- build_graph(bitcoin_df, user_lst=[], rating_type='all', maxdate='2016-01-24')
#### visualization functions:
- plot_timeline(bitcoin_df, title)
- add_user_to_graph(existing_graph, new_user, bitcoin_df)

### EDA Learnings:
alpha network does not have time level timestamp so cannot do velocity or bot analysis
or sort chronologically within a day

### Interesting Fraud Examples:
otc_user = '2680'

In [1]:
import pandas as pd
import numpy as np
import datetime
import networkx as nx
import nxpd

import matplotlib.pyplot as plt

import sys
sys.path.insert(0, '../src')
import helpers as h
import visualizations as v
import model as m

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 300)
import time

In [2]:
# Load and preprocess data
otc_df = h.load_bitcoin_edge_data('../data/soc-sign-bitcoinotc.csv.gz')
alpha_df = h.load_bitcoin_edge_data('../data/soc-sign-bitcoinalpha.csv.gz')
alpha_users = h.user_activity_dataframe(alpha_df)
otc_users = h.user_activity_dataframe(otc_df)

In [37]:
otc_df[otc_df['ratee']==4531]

,rater,ratee,rating,date,class,binomial_rating,color,penwidth
24530,4197,4531,1,2013-07-08 11:59:06,0,1,blue,1
24554,4511,4531,-10,2013-07-08 20:03:59,1,-1,red,4
24555,3719,4531,-10,2013-07-08 20:28:59,1,-1,red,4
24556,3837,4531,-10,2013-07-08 20:35:05,1,-1,red,4
24603,4522,4531,-1,2013-07-10 14:53:37,1,-1,red,1
25564,2296,4531,-10,2013-08-04 08:18:47,1,-1,red,4
25567,4622,4531,-10,2013-08-04 08:44:03,1,-1,red,4
25646,905,4531,-10,2013-08-05 12:09:38,1,-1,red,4
25733,1810,4531,-10,2013-08-05 20:10:08,1,-1,red,4
25741,3330,4531,-10,2013-08-05 20:32:10,1,-1,red,4


In [42]:
alpha_users.loc[5533]

RatingsReceived                              8
AvgRatingReceived                            0
MedianRatingReceived                         0
MinRatingReceived                          -10
MaxRatingReceived                           10
DateFirstRatingReceived    2012-09-24 21:00:00
DateLastRatingReceived     2012-10-01 21:00:00
NegReceivedCnt                               4
PosReceivedCnt                               4
RatingsGiven                                 6
AvgRatingGiven                       -0.833333
MedianRatingGiven                         -2.5
MinRatingGiven                             -10
MaxRatingGiven                              10
DateFirstRatingGiven       2012-09-24 21:00:00
DateLastRatingGiven        2012-10-01 21:00:00
NegGivenCnt                                  3
PosGivenCnt                                  3
min_ratings_delta                            0
FirstActivity              2012-09-24 21:00:00
LastActivity               2012-10-01 21:00:00
TimeActive   

In [55]:
df_otc = pd.read_csv('../data/df_features_otc.csv')
# df_otc[(df_otc['num_ratings_received']<20)&(df_otc['rating_received_sum']<0)].sort_values('ego_triad_300', ascending=False)[:15]
df_otc[df_otc['ratee']==4986]

,rater,ratee,rating,date,class,binomial_rating,color,penwidth,num_ratings_received,num_neg_received,num_pos_received,neg_ratings_pct,rating_received_sum,rating_received_avg,days_since_first_rated,days_since_last_rated,days_active,successive_neg_rating,ego_triad_300,ego_triad_210,ego_triad_201,ego_triad_120,ego_triad_all,ego_cluster_coef,ego_degree,ego_betweeness,ego_closeness,ego_num_cliques
28678,2296,4986,1,2013-11-05 16:19:37,0,1,blue,1,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0
28701,4983,4986,1,2013-11-06 16:00:20,0,1,blue,1,1.0,0.0,1.0,0.000000,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0
28707,3714,4986,1,2013-11-06 19:38:49,0,1,blue,1,2.0,0.0,2.0,0.000000,2.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.000000,4.0,1.000000,1.000000,2.0
28718,4991,4986,1,2013-11-07 08:43:21,0,1,blue,1,3.0,0.0,3.0,0.000000,3.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,2.0,0.0,3.0,0.333333,6.0,0.666667,1.000000,2.0
28721,1334,4986,1,2013-11-07 12:04:56,0,1,blue,1,4.0,0.0,4.0,0.000000,4.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,5.0,0.0,6.0,0.166667,8.0,0.833333,1.000000,3.0
28729,1810,4986,-8,2013-11-08 08:16:31,1,-1,red,4,5.0,0.0,5.0,0.000000,5.0,1.0,2.0,0.0,2.0,0.0,1.0,1.0,6.0,0.0,8.0,0.187500,9.0,0.725000,0.833333,4.0
28731,4172,4986,-10,2013-11-08 11:49:19,1,-1,red,4,6.0,1.0,5.0,0.166667,-3.0,-0.5,2.0,0.0,2.0,0.0,1.0,1.0,6.0,0.0,8.0,0.187500,9.0,0.725000,0.833333,4.0


In [3]:
# Load graph objects
_ , alpha_G = h.build_graph(alpha_df)
_ , alpha_pos_G = h.build_graph(alpha_df, rating_type='pos')
_ , otc_G = h.build_graph(otc_df)
_ , otc_pos_G = h.build_graph(otc_df, rating_type='pos')

In [ ]:
# v.plot_timeline(alpha_df, 'Alpha Bit Coin Ratings Activity')

In [ ]:
# v.plot_timeline(otc_df, 'OTC Bit Coin Ratings Activity')

## Base Features

In [ ]:
def base_metrics(bitcoin_df, user, rate_date):
    df = bitcoin_df.copy()
    
    # today's positive ratings and all prior ratings
#    user_data = df[(df['ratee']==user) & ((df['date'] < rate_date) | ((df['date']==rate_date) & (df['rating'] > 0)))]

    # all prior ratings
    user_data = df[(df['ratee']==user) & (df['date'] < rate_date)]
    
    if len(user_data)==0:
        return np.zeros(7)
    num_ratings_received = len(user_data)
    num_neg_received = user_data['class'].sum()
    num_pos_received = num_ratings_received - num_neg_received
    neg_received_pct = num_neg_received / num_ratings_received
    rating_received_sum = user_data['rating'].sum()
    rating_received_avg = user_data['rating'].mean()
    days_active = (rate_date - df[(df['ratee']==user) | (df['rater']==user)]['date'].min()).days
    return num_ratings_received, num_neg_received, num_pos_received, neg_received_pct, rating_received_sum, rating_received_avg, days_active


In [ ]:
def feature_iteration_base_metrics(bitcoin_df):
    df = bitcoin_df.copy()
    df = df[['rater', 'ratee','rating','date', 'class']]
    for i, row in df.iterrows():
        user = row['ratee']
        rate_date = row['date']
        num_ratings_received, num_neg_received, num_pos_received, neg_received_pct, rating_received_sum, rating_received_avg, days_active = base_metrics(df, user, rate_date)
        df.at[(i,'num_ratings_received')] = num_ratings_received
        df.at[(i,'num_neg_received')] = num_neg_received
        df.at[(i,'num_pos_received')] = num_pos_received
        df.at[(i,'neg_received_pct')] = neg_received_pct
        df.at[(i,'rating_received_sum')] = rating_received_sum
        df.at[(i,'rating_received_avg')] = rating_received_avg
        df.at[(i,'days_active')] = days_active
    return df   

In [ ]:
alpha_df[(alpha_df['ratee']==54) & (alpha_df['date']< '2010-11-09 21:00:00')]

In [ ]:
# base features
start_time = time.time()
df_b = feature_iteration_base_metrics(alpha_df)
print(f"{(time.time() - start_time):.0f} seconds execution time")

# Save File
df_b.to_csv('../data/df_b.csv', index=False)

## Velocity Features

In [6]:
# date velocity
start_time = time.time()
df_dv = m.feature_iteration_date_velocity(alpha_df)
print(f"{(time.time() - start_time):.0f} seconds execution time")

# Save File
df_dv.to_csv('../data/df_dv.csv', index=False)

243 seconds execution time


In [ ]:
# Retrieve File
df_dv = pd.read_csv('../data/df_dv.csv')

In [ ]:
# sequential velocity

# looks for sequential negative ratings activity
# neg_cnt_last_1_rating
# neg_cnt_last_2_rating
# neg_cnt_last_3_rating

start_time = time.time()
df_sv = m.feature_iteration_sequential_velocity(alpha_df)
print(f"{(time.time() - start_time):.0f} seconds execution time")

# Save File
df_sv.to_csv('../data/df_sv.csv', index=False)

In [ ]:
df_sv.info()

In [ ]:
# Retrieve File
df_sv = pd.read_csv('../data/df_sv.csv')

## Graph Features

- collusion features
- retalitory rating features - see node 95-188-7 (when you run 7604

In [26]:
def graph_features(bitcoin_df, user, rate_date):
    _, g = h.build_graph(bitcoin_df, rating_type='pos', maxdate=rate_date)
    if user not in g:
        return np.zeros(10)
    ego_g = nx.ego_graph(nx.reverse_view(g), user, radius=1)
    if len(ego_g) <= 2:
        return np.zeros(10)
    ego_cluster_coef = nx.clustering(ego_g, user)
    node_census = nx.triadic_census(ego_g)
    # fully closed triad (all users giving reciprocal ratings)
    ego_triad_300 = node_census['300']
    # closed triad with 2 pairs of users giving reciprocal ratings
    ego_triad_210 = node_census['210']
    # open triad with 2 pais of users giving reciprocal ratings 
    ego_triad_201 = node_census['201']
    # close triad with 1 pair of user giving reciprocal ratings
    ego_triad_120 = node_census['120U'] + node_census['120D'] + node_census['120C']
    ego_triad_all = triad_300 + triad_210 + triad_201 + triad_120
    ego_degree = nx.degree(ego_g)[user]
    # centrality measures
    ego_betweeness = nx.betweenness_centrality(ego_g)[user]
    ego_closeness = nx.closeness_centrality(ego_g)[user]
    ego_g = ego_g.to_undirected()
    ego_num_cliques = nx.number_of_cliques(ego_g, user)

    return ego_cluster_coef, triad_300, triad_210, triad_201, triad_120, triad_all, ego_degree, ego_betweeness, ego_closeness, num_ego_cliques

## Networkx Functions

In [ ]:
alpha_GU = alpha_G.to_undirected()
nx.number_of_cliques(alpha_GU, nodes=7551)

In [ ]:
## Run the triadic census
census = nx.triadic_census(alpha_G)
census

In [ ]:
# triad generator
triads = all_triads(alpha_G)

In [ ]:
node_census = nx.triads_by_type(alpha_G)

In [ ]:
keys=node_census.values()[1].keys() 

## Generate a table header
print('| Node |', ' | '.join(keys))
for k in node_census.keys(): 
     print('|', k, '|',' | '.join([str(v) for v in node_census[k].values()]))


In [ ]:
node_census

In [ ]:
len(census)

In [ ]:
# Connected_component_subgraphs() returns a list of components,
# sorted largest to smallest
components=nx.connected_component_subgraphs(alpha_G)
# pick the first and largest component
cc = components[0]

In [ ]:
from networkx.algorithms import community

communities_generator = community.girvan_newman(alpha_G)
top_level_communities = next(communities_generator)
next_level_communities = next(communities_generator)

sorted(map(sorted, top_level_communities))



The Girvan–Newman algorithm detects communities by progressively removing edges from the original graph. The algorithm removes the “most valuable” edge, traditionally the edge with the highest betweenness centrality, at each step. As the graph breaks down into pieces, the tightly knit community structure is exposed and the result can be depicted as a dendrogram.

In [ ]:
import networkx.generators.small

g = networkx.generators.small.krackhardt_kite_graph()

g.adjacency_list()

In [ ]:
otc_users[otc_users['BotActivity']==True].sort_values('TimeActive')

# Need to create visualization to Define Bot acitivity
In Alpha network only bot activity is with zero time delay - all same day raters
In OTC network only 18 users have zero time delay, however, XXX
users have delay under 1? minute (what is delay )

need 3 eda plots of same day rater distribution - day by hour, hour by minute, minute by second

In [ ]:
print(f"Alpha Bot users: {alpha_users['BotActivity'].sum()}")
print(f"OTC Bot users: {otc_users['BotActivity'].sum()}")

# Prediction Visualizations

In [ ]:
df = pd.read_csv("../data/alpha_with_ratee_stats.csv")

In [ ]:
df_ato = m.feature_iteration_ato(alpha_df)

In [ ]:
df_ato.reset_index(drop=True, inplace=True)
df_ato.head()

In [ ]:
df = pd.concat([df, df_ato], axis=1)

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
X = df.copy()
X = X.drop(['rater', 'ratee', 'rating','date', 'color', 'penwidth', 'binomial_rating'], axis=1)
y = X.pop('class')


X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    stratify=y, shuffle=True,
                                                    random_state=123)

RF = RandomForestClassifier(n_jobs=-1, random_state=123)
RF.fit(X_train, y_train)
y_preds = RF.predict(X_test)
recall = recall_score(y_test, y_preds)
precision = precision_score(y_test, y_preds)

# print(X_test[(y_preds==0) & (X_test['num_neg_received']>0)].head(10))
print(recall)
print(precision)
RF.feature_importances_
 

In [ ]:
import seaborn as sns
fig = plt.figure(figsize=(15,15))

name = "Random Forest"
indices = np.argsort(RF.feature_importances_)[::-1][:40]
ax1 = sns.barplot(y=X_train.columns[indices][:40],x = RF.feature_importances_[indices][:40] , orient='h')
ax1.set_xlabel("Relative importance",fontsize=12)
ax1.set_ylabel("Features",fontsize=12)
ax1.tick_params(labelsize=9)
ax1.set_title(name + " feature importance")

### Confusion Matrix

In [ ]:
from sklearn.metrics import confusion_matrix

thresh = 0.2
pred_proba = RF.predict_proba(X_test)[:,1]
cnf_matrix = confusion_matrix(y_test, pred_proba>=thresh)
print(cnf_matrix)
tn, fp, fn, tp = cnf_matrix.ravel()
tn, fp, fn, tp
neg = tn + fp
pos = fn + tp
tnpct = tn/neg
fppct = fp/neg
fnpct = fn/pos
tppct = tp/pos
cnt_matrix_pct = np.round(np.array([tnpct,fppct,fnpct, tppct]), 2)
cnt_matrix_pct.reshape((2,2))

In [ ]:
df.iloc[y_test-y_preds].shape

In [ ]:
y_test-y_preds

In [ ]:
X_test.loc[22765]

In [ ]:
df.loc[22765]

In [ ]:
for input, prediction, label in zip(X_test, y_preds, y_test):
    if prediction != label:
        print(input, 'has been classified as ', prediction, 'and should be ', label) 

In [ ]:
def plot_confusion_matrix(ax, cm, title, classes=['Legitimate','Fraud'],
                          cmap=plt.cm.Blues, currency=False):
    """
    Plots a single confusion matrix. If currency=True then displays results as currency.

    Parameters
    ----------
    cm: array (confusion matrix)
    title: String
    test_size: float - size/percentage of holout dataset
    goal: float - project goal for ultimate dollar loss rate

    Returns
    -------
    """   
    
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        cost=cm[i, j]
        if currency:
            cost = f'${cost:0,.2f}' 
        ax.text(j, i, cost, horizontalalignment="center", 
        color="white" if cm[i, j] > thresh else "black")
    ax.imshow(cm, interpolation='nearest', cmap=cmap)

    if currency:
        ax.set_title(f'{title}\nCost Matrix')
    else:
        ax.set_title(f'{title}\nConfusion Matrix')
    tick_marks = np.arange(len(classes))
    ax.set_xticks(tick_marks)
    ax.set_xticklabels(classes, rotation=0)
    ax.set_yticks(tick_marks)
    ax.set_yticklabels(classes, rotation=90)

    ax.set_xlabel('Predicted')
    ax.set_ylabel('Actual')    



In [ ]:
import itertools
fig, ax = plt.subplots()
plot_confusion_matrix(ax, cnf_matrix, "test", classes=['Pos Rating','Neg Rating'])

In [ ]:
from sklearn.metrics import precision_recall_curve

# Make function for indivisual plot and then you can call it over and over
def Plot_PR_Curve(X, y):
    '''
    Calculates and Plots PR Curve and puts threshold marker on curve.
    '''
    # get points to plot on the PR Curve
    thresh = 0.5
    precision, recall, thresholds = precision_recall_curve(y, pred_proba) 
        
    # find the index of the record with closest threshold to desired threshold value
    threshold_idx = np.argmin(np.abs(thresholds-thresh))
    
    plt.plot(precision, recall)
    plt.plot(precision, threshold_idx, recall, threshold_idx, 'o', 
             markersize=10, fillstyle='full', 
             label="{thresh:.2f} threshold", mew=2)

    plt.legend(loc='center', frameon=False)
    plt.title('Precision-Recall Curve Comparison')    
    plt.xlabel('Precision')
    plt.ylabel('Recall')
    plt.xlim(xmin=-0.05,xmax=1.05);
    plt.ylim(ymin=-0.05,ymax=1.05);  

In [ ]:
Plot_PR_Curve(X_test, y_test)

## Node2vec

In [ ]:
# embedding dimension to 14, the number of walks to 25, and the number of iterations to 15.

from node2vec import Node2Vec

# Precompute probabilities and generate walks
# node2vec = Node2Vec(G, dimensions=64, walk_length=30, num_walks=200, workers=4)
node2vec = Node2Vec(G, dimensions=14, walk_length=30, num_walks=25, workers=4)

# Embed
# windows is the number of max distance from the node that the vector is going to be based on
# maybe i can move this to 1 or 2??
model = node2vec.fit(window=5, min_count=1)#, batch_words=4)  # Any keywords acceptable by gensim.Word2Vec can be passed, `diemnsions` and `workers` are automatically passed (from the Node2Vec constructor)

# Look for most similar nodes
model.wv.most_similar('2')  # Output node names are always strings

# giving me 10 most similar to user
user = '1006'
model.wv.most_similar(user) 

# gives 64 length vector for user
vector = model.wv[user]
print(vector)

embeddingsframe = pd.DataFrame(model.get_embedding())

n = [] # node list?
e = [] # embeddings list

with open('./trimmed_network.emb') as fin:
    for line in model:
        node_emb = line.strip().split()  # turns into a list and removes white spaces at beginning and ending of string
        n.append(node_emb[0])
        e.append(node_emb[1:])

n = n[1:]
n = [int(i) for i in n] #  converts node to an int datatype
embs = np.zeros([len(e)-1,14])
for i in range(1,len(e)):
    embs[i-1] = e[i]
embs.shape

# Save embeddings for later use
model.wv.save_word2vec_format(EMBEDDING_FILENAME)

# Save model for later use
model.save(EMBEDDING_MODEL_FILENAME)

In [ ]:
from matplotlib.dates import DateFormatter, DayLocator

df2 = df[df['ratee']==7512].copy()
df2.set_index('date', inplace=True)
fig, ax = plt.subplots(figsize=(16,4))
ax.bar(df2.index.values,
       df2['vel_24_all'],
       color='purple')
# ax = df2['vel_24_all'].plot(color='r', kind='bar', label='vel');

In [ ]:
df[(alpha_df['rater']==7512) | (df['ratee']==7512)].head()#[['date','vel_24_all','vel_24_in_all','vel_24_out_all']]

In [ ]:
alpha_df.groupby(['rater', 'date'])['rating'].count().sort_values(ascending=False)

In [ ]:
vel_24_in_neg

In [ ]:
alpha_df[(alpha_df['rater']==185) | (alpha_df['ratee']==185)]